In [8]:
import Data
import nltk
from Models import nGrams as n

In [2]:
poems = Data.PoetryFoundationPoems()
poems = Data.Poem_words_dict(poems)

In [3]:
def word_pos_tags(dic, poems_tagged):
    for word, tag in poems_tagged:
        if(word in dic):
            if(tag in dic[word]): 
                dic[word][tag] = dic[word][tag]+1
            else:
                dic[word][tag] = 1
        else:
            dic[word] = {tag:1}
    return dic

def tag_pos(poem, dic):
    poem = [w.lower() for w in poem]
    nltk_tags = nltk.pos_tag(poem)
    modify_tags = []
    only_tags = []
    for word,tag in nltk_tags:
        if(word == "newline"):
            modify_tags = modify_tags + [(word,"NEWLINE")]
            only_tags = only_tags + ["NEWLINE"]
        else: 
            modify_tags = modify_tags + [(word,tag)]
            only_tags = only_tags + [tag]

    dic = word_pos_tags(dic, modify_tags)
    #print(modify_tags)
    #print()
    return(only_tags, dic)

In [432]:
poems_of_tags = []
pos_dicts = {}
for p in poems:
    poem_tagged, pos_dicts = tag_pos(p, pos_dicts)
    poems_of_tags = poems_of_tags + [poem_tagged]
    #print(pos_dicts)

In [9]:
a = Models.nGrams()

In [465]:
WordsTri = {}
for i in poems:
    WordsTri = a.NGram(WordsTri, i,3)

Words4 = {}
for i in poems:
    Words4 = a.NGram(Words4, i, 4)

In [66]:
POS3 = {}
for i in poems_of_tags:
    POS3 = a.NGram(POS3, i, 3)

POS4 = {}
for i in poems_of_tags:
    POS4 = a.NGram(POS4, i, 4)

In [52]:
n.next_word_prob(["HE","IS"], WordsTri, 3, 0)

('NOT', 0.06657963446475196)

In [53]:
WordsTri["HE"]["IS"]

{'SO': 0.007832898172323759,
 'BEAUTIFUL': 0.0026109660574412533,
 'SMALL': 0.0013054830287206266,
 'YOURS': 0.0013054830287206266,
 'STUDYING': 0.0013054830287206266,
 'MORE': 0.0039164490861618795,
 'CONCENTRATING': 0.0013054830287206266,
 'DECORATING': 0.0013054830287206266,
 '.': 0.0195822454308094,
 'DELIGHTED': 0.0013054830287206266,
 'SORRY': 0.0013054830287206266,
 'RUNNING': 0.0013054830287206266,
 'THE': 0.06135770234986945,
 'A': 0.05221932114882506,
 'CORPORATE': 0.0013054830287206266,
 'DUNKIN': 0.0013054830287206266,
 'NIKE': 0.0013054830287206266,
 'FORGED': 0.0013054830287206266,
 'PASSING': 0.0013054830287206266,
 'STUTTER': 0.0013054830287206266,
 'NEWLINE': 0.03263707571801567,
 'SPEAKING': 0.0026109660574412533,
 'GIVING': 0.0013054830287206266,
 'WEARING': 0.0039164490861618795,
 'BIGGIE': 0.0013054830287206266,
 'MILES': 0.0013054830287206266,
 'AKRON': 0.0013054830287206266,
 'HEART': 0.0013054830287206266,
 'ASHTABULA': 0.0013054830287206266,
 'CARRYING': 0.0052

In [19]:
n.next_word_prob(["NN","VB"], POS3, 3, 0)

('PRP', 0.12964824120603016)

In [433]:
from Models import nGrams as nn

def nextPOSWord(words, POS_tags_of_words, words_nGram, POS_nGram, POS_dic_words, n, a=0.2):    
    wordsDic    = words_nGram
    POS_Ngram   = POS_nGram
    
    nn.next_word_prob(words, wordsDic, n, a)
    nn.next_word_prob(POS_tags_of_words, POS_Ngram, n, a)

    for word in words:
        wordsDic = wordsDic[word]
        POS_prob(word.lower(), POS_dic_words, a)
        
    for POS_tag in POS_tags_of_words:
        POS_Ngram = POS_Ngram[POS_tag]
        
    Max = 0
    next_word = "Error"
    next_tag  = "Error2"
    for word,probablity_word in list(wordsDic.items()):
        POS_prob(word.lower(), POS_dic_words, a)
        word_POS_p = list(POS_dic_words[word.lower()].items())
        #print(word_POS_p)
        for tag, prob_tag_given_word in word_POS_p:
            final_prob = 0
            #print(tag, prob_tag_given_word)
            if(tag in POS_Ngram):
                prob_tag_for_tag_sequence = POS_Ngram[tag]
                final_prob = probablity_word * prob_tag_given_word*prob_tag_for_tag_sequence
            
            #print(word,probablity_word,syllaDic[word_syllable_count], final_prob)
            if(final_prob>Max):
                Max = final_prob
                next_word = word
                next_tag  = tag
    #print(next_word, Max)
    return(next_word, next_tag)


In [431]:
nextPOSWord(["WHY","DO"],["WRB","VBP"],WordsTri,POS3,pos_dicts,3 )

('YOU', 'PRP')

In [400]:
POS_prob(".",pos_dicts, a = 0)

In [402]:
pos_dicts["."]

{'.': 1.0}

In [395]:
def POS_prob(word, dic, a = 0.2):
    counts_dic = dic[word]
    
    if(not (((list(counts_dic.values())[0])<1)) ):
        total_counts = 0
        for POS_tag,Count in list(counts_dic.items()):
            total_counts = total_counts+Count
        
        items = list(counts_dic.items())
        alpha = a*total_counts
        for POS_tag,Count in items:
            counts_dic[POS_tag] = ((Count+alpha)/(total_counts + alpha*len(items)))

In [396]:
pos_dicts["do"]

{'VBP': 0.666610455311973,
 'VB': 0.32377740303541314,
 'JJ': 0.004047217537942664,
 'NNS': 0.0003372681281618887,
 'NN': 0.00505902192242833,
 'FW': 0.00016863406408094435}

In [397]:
pos_dicts["not"]

{'RB': 1.0}

In [469]:
words44 = Words4.copy()
POS44   = POS4.copy()
dic_word = pos_dicts.copy()

In [473]:
Words4["WHY"]["DO"]["I"]

{'FEEL': 0.04404761904761905,
 'HAVE': 0.04404761904761905,
 'OVERLIVE': 0.04285714285714286,
 'BELIEVE': 0.04285714285714286,
 'DO': 0.04404761904761905,
 'NEED': 0.04642857142857143,
 'THINK': 0.04285714285714286,
 'TELL': 0.04523809523809524,
 'SEE': 0.04285714285714286,
 'ONLY': 0.04285714285714286,
 'LIVE': 0.04285714285714286,
 'WEEP': 0.04285714285714286,
 'SIGH': 0.04285714285714286,
 'WELCOME': 0.04285714285714286,
 'TAKE': 0.04285714285714286,
 'SUFFER': 0.04285714285714286,
 'WRITE': 0.04285714285714286,
 'ALWAYS': 0.04285714285714286,
 'DREAM': 0.04285714285714286,
 'HOPETO': 0.04285714285714286,
 'HATE': 0.047619047619047616,
 'ASK': 0.04285714285714286,
 'KNOW': 0.04285714285714286}

In [499]:
prev_word1 = "IS"#words_by_syllables[2][random.randint(0,len(words_by_syllables[3]))]
prev_tag1  = "VBZ"
prev_word2 = "IT"#temp[random.randint(0,len(temp)-1)]
prev_tag2  = "DT"
prev_word3 = "REAL"#temp2[random.randint(0,len(temp2)-1)]
prev_tag3  = "JJ"

generate_str = prev_word1+" "+prev_word2 + " " +prev_word3
print(generate_str)
print()
for i in range(100):
    #print(generate_str)
    next_words,next_tag = nextPOSWord([prev_word1, prev_word2, prev_word3],
                             [prev_tag1 , prev_tag2 , prev_tag3 ],
                             words44, POS44,dic_word,4, a = 0.5)
    prev_word1 = prev_word2
    prev_word2 = prev_word3
    prev_word3 = next_words
    prev_tag1  = prev_tag2
    prev_tag2  = prev_tag3
    prev_tag3  = next_tag
    generate_str = generate_str+" "+ next_words
print(generate_str.replace("NEWLINE ","\n"))

IS IT REAL

IS IT REAL 
THE BOTTLE OF THE EYE , 
SOURCE OF VIGOR 110 YEARS OLD . 
DUSK FALLS , AND A MAN IN A MOMENT OF THE WORLD , 
SOMETHING BETWEEN SONG AND PRAYER TO DO SOMETHING , 
ANYTHING , PAIN , PAIN EVER , FOR EVER , 
TIME FOR THE PRESUMPTIONSOF MICE AND MEN 
WITH THEIR MOUSSE BACKED HAIR AND CRAYOLA 
BLACK COATS LIKE SMALL FRY HOODS WHO SMOKE 
AND FIRE ABOVE HIS HEAD , 
SOMETHING BETWEEN SONG AND PRAYER TO DO SOMETHING , 
ANYTHING , PAIN , PAIN EVER ,


In [355]:
pos_dicts["know"]

{'VBP': 0.6035981402870426,
 'VB': 0.37234687689508794,
 'NN': 0.009702850212249849,
 'VBD': 0.0028299979785728724,
 'NNS': 0.003840711542348898,
 'JJ': 0.00505356781888013,
 'FW': 0.0008085708510208206,
 'VBN': 0.0004042854255104103,
 'RB': 0.00020214271275520516,
 'IN': 0.00020214271275520516,
 'CD': 0.00020214271275520516,
 'VBZ': 0.0006064281382656155,
 'NNP': 0.00020214271275520516}

In [390]:
prev_word1 = "IS"#words_by_syllables[2][random.randint(0,len(words_by_syllables[3]))]
prev_tag1  = "VBZ"
temp       = list(Words4[prev_word1].keys())
prev_word2 = "IT"#temp[random.randint(0,len(temp)-1)]
prev_tag2  = "DT"
temp2      = list(Words4[prev_word1][prev_word2].keys())
prev_word3 = "REAL"#temp2[random.randint(0,len(temp2)-1)]
prev_tag3  = "RB"
prev_word4 = "KNOW"#temp2[random.randint(0,len(temp2)-1)]
prev_tag4  = "VB"

next_words,next_tag = nextPOSWord([prev_word1, prev_word2, prev_word3],
                             [prev_tag1 , prev_tag2 , prev_tag3 ],
                             words44, POS44,pos_dicts,4)
print(next_words, next_tag)

next_words,next_tag = nextPOSWord([prev_word2, prev_word3, prev_word4],
                             [prev_tag2 , prev_tag3 , prev_tag4 ],
                             words44, POS44,pos_dicts,4)
print(next_words, next_tag)

KNOW VB
. .


In [403]:
prev_word1 = "I"#words_by_syllables[2][random.randint(0,len(words_by_syllables[3]))]
prev_tag1  = "NN"
temp       = list(Words4[prev_word1].keys())
prev_word2 = "DO"#temp[random.randint(0,len(temp)-1)]
prev_tag2  = "VBP"
temp2      = list(Words4[prev_word1][prev_word2].keys())
prev_word3 = "NOT"#temp2[random.randint(0,len(temp2)-1)]
prev_tag3  = "RB"

next_words,next_tag = nextPOSWord([prev_word1, prev_word2, prev_word3],
                             [prev_tag1 , prev_tag2 , prev_tag3 ],
                             words44, POS44,pos_dicts,4)
print(next_words, next_tag)


KNOW VB


In [496]:
nltk.pos_tag(["is","this","real","life","or","is","it"])

[('is', 'VBZ'),
 ('this', 'DT'),
 ('real', 'JJ'),
 ('life', 'NN'),
 ('or', 'CC'),
 ('is', 'VBZ'),
 ('it', 'PRP')]

In [412]:
from Models import nGrams as n

prev_word1 = "I"
prev_word2 = "DO"#n.next_word([prev_word1], dicBi, 2)
prev_word3 = "NOT"#n.next_word([prev_word1,prev_word2], dicTri, 3)
generate_str = prev_word1+" "+prev_word2 + " " +prev_word3

for i in range(50):
    next_word = n.next_word([prev_word1, prev_word2,prev_word3], Words4, 4)
    prev_word1 = prev_word2
    prev_word2 = prev_word3
    prev_word3 = next_word
    generate_str = generate_str+" "+ next_word

print(generate_str.replace("NEWLINE ","\n"))

I DO NOT KNOW 
HOW TO MAKE THE WORLD SAFE FOR DEMOCRACY , 
YOU ’ RE A LITTLE BIT 
ABOUT THE FLOWERS IN THE STALLS , AND THE LONG , LONG THOUGHTS . `` I REMEMBER THE FIRST TIME , 
AND THE WORLD IS A ROAD UNDER THE WALL


In [21]:
print(nltk.pos_tag(["children","played","slow"]))

[('children', 'NNS'), ('played', 'VBD'), ('slow', 'NN')]
